In [1]:
import pickle
import json
import os

In [2]:
from rouge import Rouge
import json
from tqdm import tqdm
def compute_rouge_l(batch_data):
    """
    输入为一个元组list，每一个元组是一个pred/gt对
    Compute the average ROUGE-L F1 score for a batch of predictions and ground truths.

    Args:
        batch_data (list): A list of tuples where each tuple contains:
            - prediction (str): The model's predicted text.
            - ground_truth (str): The reference ground truth text.

    Returns:
        float: The average ROUGE-L F1 score over the batch.
    """
    # Initialize the ROUGE evaluator with only ROUGE-L metric
    rouge = Rouge(metrics=['rouge-l'])

    # Separate predictions and ground truths
    predictions = [pred for pred, gt in batch_data]
    ground_truths = [gt for pred, gt in batch_data]
    try:
    # Compute ROUGE-L scores for the batch
        scores = rouge.get_scores(predictions, ground_truths, avg=True)
    except Exception as e:
        print(e)
    # Extract the average ROUGE-L F1 score
    rouge_l_f1 = scores['rouge-l']['f']

    return rouge_l_f1

def eval_samsum(data,evaled_model_name):
    eval_tuple_list = [(data_i[f'{evaled_model_name}_res'], data_i['summary']) for data_i in data]
    #eval_tuple_list = [(data_i[f'refine_res'], data_i['summary']) for data_i in data]
    print(eval_tuple_list)
    rougl = compute_rouge_l(eval_tuple_list)
    print(rougl)
    return rougl

def eval_qnli(data,evaled_model_name):
    right_num = 0
    total = 0
    for i in data:
        #check
        mapping = {
            'entailment': 0,
            'not entailment': 1,
        }
        if i[f'{evaled_model_name}_res'] not in ['entailment','not entailment']:
            raise ValueError("QNLI数据集有问题")

        if i['label'] == mapping[i[f'{evaled_model_name}_res']]:
            right_num = right_num+1
            total = total+1
        else:
            total = total+1
    print("acc: "+str(right_num/total))
    return right_num/total

def eval_reclor(data,evaled_model_name):
    right_num = 0
    total = 0
    map_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    for i in data:
        if i[f'{evaled_model_name}_res'] in ['A','B','C',"D"]:
            if i[f'{evaled_model_name}_res'] == chr(i['label']+65):
                right_num = right_num+1
                total = total+1                
            else:
                total = total+1
        else:
            continue
    print("acc: "+str(right_num/total))
    return right_num/total

def eval_one_file(data,dataset_name, evaled_model_name):
    print(f'开始评估数据集: {evaled_model_name}-{dataset_name}')
    if dataset_name == "SAMsum":
        return eval_samsum(data,evaled_model_name)
    elif dataset_name == "QNLI":
        return eval_qnli(data,evaled_model_name)
    elif dataset_name == "reClor":
        return eval_reclor(data,evaled_model_name)

In [3]:
import re
def find_pii_leakage(text,word_set):

    #words_in_text = re.findall(r'\b\w+\b', text)
    matched_words = []
    for word in word_set:
        if word in text:
            matched_words.append(word)
    #matched_words = {word for word in words_in_text if word in word_set}
    return matched_words,len(matched_words)

def routine_san(data,task_name):
    real_pii_path = f"./tak/extract_out/{task_name}/{task_name}.json"
    #data_path = f"./eval_data/{privacy_method}/{task_name}/{task_name}.json"
    # with open(data_path, 'r') as f:
    #     data = json.load(f)
    with open(real_pii_path, 'r') as f:
        real_pii = json.load(f)
    pii_list = []
    for key in real_pii:
        pii_list = pii_list+real_pii[key]
    pii_set = set(pii_list)
    total_leakage = 0
    corpus = ""
    for data_i in data:
        if task_name == "SAMsum":
            corpus = f"{corpus} {data_i['dialogue']}"
        if task_name == "QNLI":
            corpus = f"{corpus} {data_i['text1']} {data_i['text2']}"
        if task_name == "reClor":
            corpus = f"{corpus} {data_i['context']} {data_i['question']} {str(data_i['answers'])}"
    leak_set,leak_num = find_pii_leakage(corpus,pii_set)
    total_leakage = leak_num
    return 1-(total_leakage/len(pii_set))

def routine(task_name, privacy_method,eps):
    real_pii_path = f"./tak/extract_out/{task_name}/{task_name}.json"
    data_path = f"./eval_data/{privacy_method}/{task_name}/eps_{str(eps)}/{task_name}.json"
    with open(data_path, 'r') as f:
        data = json.load(f)
    with open(real_pii_path, 'r') as f:
        real_pii = json.load(f)
    
    pii_list = []
    for key in real_pii:
        pii_list = pii_list+real_pii[key]
    pii_set = set(pii_list)
    total_leakage = 0
    corpus = ""
    for data_i in data:
        if task_name == "SAMsum":
            corpus = f"{corpus} {data_i['dialogue']}"
        if task_name == "QNLI":
            corpus = f"{corpus} {data_i['text1']} {data_i['text2']}"
        if task_name == "reClor":
            corpus = f"{corpus} {data_i['context']} {data_i['question']} {str(data_i['answers'])}"


    leak_set,leak_num = find_pii_leakage(corpus,pii_set)
    total_leakage = leak_num
    return 1-(total_leakage/len(pii_set))

In [5]:
dataset_name = "SAMsum"

In [9]:
our_san_data_path_list = [
    # f"./dataset/code_sanitization/SPO/0.3_0.1/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.5_0.1/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.7_0.1/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.9_0.1/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.5_0.3/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.7_0.3/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.9_0.3/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.7_0.5/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.9_0.5/graph/{dataset_name}.json",
    # f"./dataset/code_sanitization/SPO/0.9_0.7/graph/{dataset_name}.json",

    #无隐私处理+语义图
    #f"./dataset/no/SPO/0.3_0.1/graph/{dataset_name}.json"
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_1.0\\{dataset_name}.json",
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_2.0\\{dataset_name}.json",
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_3.0\\{dataset_name}.json",
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_4.0\\{dataset_name}.json",
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_5.0\\{dataset_name}.json",
    f"E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\{dataset_name}\\eps_6.0\\{dataset_name}.json",
]


In [8]:
res_list = []
for path in our_san_data_path_list:
    with open(path, "r") as f:
        data = json.load(f)
    res_list.append((path,eval_one_file(data, dataset_name, "gpt-3.5-turbo"),routine_san(data,dataset_name)))
res_list

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\desktop\\prompt-privacy-final\\eval_data\\icl\\QNLI\\eps_1.0\\SAMsum.json'